In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/329T/Project/

[Errno 2] No such file or directory: '/content/drive/Shareddrives/329T/Project/'
/content


In [ ]:
%cd /content/drive/MyDrive/329T Project/

/content/drive/MyDrive/329T Project


In [ ]:
# Install packages 
!pip3 install transformers 
!pip3 install torch

In [ ]:
import warnings
import os

import torch
import torch.nn as nn
import json
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from torch.nn.functional import cosine_similarity
from transformers import BertConfig, BertTokenizer, AutoConfig, AutoTokenizer, DistilBertConfig, DistilBertTokenizer
from transformers import BertForSequenceClassification, BertForMaskedLM, AutoModelForSequenceClassification, DistilBertForMaskedLM
import copy
import argparse
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import re
import copy

In [ ]:
import sys
import importlib
import subprocess
from pathlib import Path

def install_if_not_installed(packages):
  """Install the given packages if they are not already installed."""

  for package in packages:
    if isinstance(package, tuple):
      package_name, package_package = package
    else:
      package_name = package
      package_package = package

    print(f"{package_name} ... ", end='')

    try:
      importlib.import_module(package_name)
      print("already installed")

    except:
      print(f"installing from {package_package}")
      subprocess.check_call(
          [sys.executable, "-m", "pip", "install", package_package]
      )

install_if_not_installed(
    [("trulens", "git+https://github.com/truera/trulens.git@piotrm/vis/output-detect"),
     "pandas",
     "numpy",
     "domonic"]
)

trulens ... already installed
pandas ... already installed
numpy ... already installed
domonic ... already installed


In [ ]:
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.models import get_model_wrapper
from trulens.nn.attribution import Cut, OutputCut
from trulens.nn.quantities import MaxClassQoI, ClassQoI, ComparativeQoI
from torch.nn.functional import cosine_similarity
from trulens.visualizations import NLP

##### Args

In [ ]:
device = 'cuda'
data_path = "tweet_sentiment_pairs.csv"
mlm_path = "distilbert-base-uncased"
tgt_path = "distilbert-base-uncased-finetuned-sst-2-english"
output_dir = "perturbations.csv"
use_sim_mat = 0 
start = 0
end = 100
num_label = 2 
use_bpe = 1
k = 5
threshold_pred_score = 0

##### Initialize models

In [ ]:
tokenizer_mlm = DistilBertTokenizer.from_pretrained(mlm_path, do_lower_case=True)
tokenizer_tgt = AutoTokenizer.from_pretrained(tgt_path, do_lower_case=True)

config_atk = DistilBertConfig.from_pretrained(mlm_path)
mlm_model = DistilBertForMaskedLM.from_pretrained(mlm_path, config=config_atk)
mlm_model.to(device)

config_tgt = AutoConfig.from_pretrained(tgt_path, num_labels=num_label)
tgt_model = AutoModelForSequenceClassification.from_pretrained(tgt_path, config=config_tgt)
tgt_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# For pytorch, input_shape is required... I just used the tokenizer max_length here
wrapped_model = get_model_wrapper(tgt_model, input_shape=(None, 128), device=device)

INFO: Detected pytorch backend for <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>.
INFO: Using backend Backend.PYTORCH.
INFO: If this seems incorrect, you can force the correct backend by passing the `backend` parameter directly into your get_model_wrapper call.


### Integrated Gradients

In [ ]:
def tokenize(inputs):
    return tokenizer_tgt(inputs, padding=True, return_tensors="pt").to(device)
    # pt refers to pytorch tensor

In [ ]:
# Baseline 
from trulens.utils.nlp import token_baseline

inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
    keep_tokens=set([tokenizer_tgt.cls_token_id, tokenizer_tgt.sep_token_id]),
    # Which tokens to preserve.

    replacement_token=tokenizer_tgt.pad_token_id,
    # What to replace tokens with.

    input_accessor=lambda x: x.kwargs['input_ids'],

    ids_to_embeddings=tgt_model.get_input_embeddings()
    # Callable to produce embeddings from token ids.
)


# Provisional baseline for calling .attributions as opposed to visualizing with NLP object
inputs_baseline_ids_attribution, inputs_baseline_embeddings_attribution = token_baseline(
    keep_tokens=set([tokenizer_tgt.cls_token_id, tokenizer_tgt.sep_token_id]),
    # Which tokens to preserve.

    replacement_token=tokenizer_tgt.pad_token_id,
    # What to replace tokens with.

    input_accessor=lambda x: x.args[0],

    ids_to_embeddings=tgt_model.get_input_embeddings()
    # Callable to produce embeddings from token ids.
)

In [ ]:
def similarity_metrics(x,y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    diff = x - y
    return {
        'L_1': torch.norm(diff, p=1).item(),
        'L_2': torch.norm(diff, p=2).item(),
        'L_inf': torch.max(torch.abs(diff)).item(),
        'cosine_sim': cosine_similarity(x,y,dim=0).item()
    }

IG_calc = IntegratedGradients(
    wrapped_model, 
    baseline=inputs_baseline_embeddings, 
    resolution=50,
    doi_cut=Cut('distilbert_embeddings_word_embeddings'), 
    qoi=ComparativeQoI(1, 0), #qoi='max',
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

IG_calc_attribution = IntegratedGradients(
    wrapped_model, 
    baseline=inputs_baseline_embeddings_attribution, 
    resolution=50,
    doi_cut=Cut('distilbert_embeddings_word_embeddings'), 
    qoi=ComparativeQoI(1, 0), #qoi='max',  
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

In [ ]:
'''
Input:  integrated gradient calculator: (trulens.nn.attribution.IntegratedGradients())
        original text: (str)
        perturbed text: (str)
Output: attribution for original text: (tensor of shape (?))
        attribution for perturbed text: (tensor of shape (?))
Function:
        encode raw text of an original-adversarial pair into input tokens for the distilbert model
        feed through IG
        compute metrics
'''
def calculate_IGs(IG, string):
    encode_string = tokenize(string)
    input_ids = torch.tensor(encode_string["input_ids"])
    attribution = IG.attributions(input_ids.to(device))
    
    return attribution

### Finding a suitable distance + threshold

In [ ]:
perturbed_data = pd.read_csv('enriched_data.csv')

In [ ]:
test_original = perturbed_data['seq_a'][0]
test_adversary = perturbed_data['adv'][0]

explanations = calculate_IGs(IG_calc_attribution, test_original, test_adversary)
print(test_original)
print(test_adversary)
print(explanations[0].shape, explanations[1].shape)
if explanations[0].shape == explanations[1].shape:
    metrics = similarity_metrics(explanations[0].flatten(), explanations[1].flatten())
    print("Metrics:", metrics)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
awww, that's a bummer.  you shoulda got david green of second morning to do it. ;d
torch.Size([1, 28, 768]) torch.Size([1, 28, 768])
Metrics: {'L_1': 30.263, 'L_2': 0.439, 'L_inf': 0.058, 'cosine_sim': 0.928}


In [ ]:
from trulens.visualizations import NLP

V = NLP(
    wrapper=wrapped_model,
    labels=['negative', 'positive'],
    decode=lambda x: tokenizer_tgt.decode(x),
    tokenize=tokenize,
    # huggingface models can take as input the keyword args as per produced by their tokenizers.

    input_accessor=lambda x: x['input_ids'],
    # for huggingface models, input/token ids are under input_ids key in the input dictionary

    output_accessor=lambda x: x['logits'],
    # and logits under 'logits' key in the output dictionary

    hidden_tokens=set([tokenizer_tgt.pad_token_id])
    # do not display these tokens
)

# display(
#     V.tokens(sentences, attributor=IG_calc)
# )

In [ ]:
pairs = list(zip(list(perturbed_data['seq_a']), list(perturbed_data['adv'])))

In [ ]:
original_attributions = []
perturbed_attributions = []
L1s = []
L2s = []
Linfs = []
cos_sims = []

for i in range(len(pairs)):
    pair = pairs[i]
    original_attribution = calculate_IGs(IG_calc_attribution, pair[0])
    perturbed_attribution = calculate_IGs(IG_calc_attribution, pair[1])
    if original_attribution.shape == perturbed_attribution.shape:
        print("--------------------------------")
        print(f"Iteration: {i}")
        print("Original: ", pair[0])
        print("Adversary: ", pair[1])
        orig_attr_summed = torch.sum(original_attribution, -1).flatten().cpu()
        pert_attr_summed = torch.sum(perturbed_attribution, -1).flatten().cpu()
        original_attributions.append(orig_attr_summed.cpu())
        perturbed_attributions.append(pert_attr_summed.cpu())
        # print(orig_attr_summed)
        # print(pert_attr_summed)

        # metrics = similarity_metrics(orig_attr_summed, pert_attr_summed)
        # L1s.append(metrics["L_1"])
        # L2s.append(metrics["L_2"])
        # Linfs.append(metrics["L_inf"])
        # cos_sims.append(metrics["cosine_sim"])
        display(V.tokens(list(pair), attributor=IG_calc))
        print("--------------------------------")
        
        # print("Metrics:", metrics)
    else:
        
        print(f"Token number mismatch on pair {i}: {original_attribution.shape} != {perturbed_attribution.shape}")
        assert 1 == 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


--------------------------------
Iteration: 0
Original:  Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Adversary:  awww, that's a bummer.  you shoulda got david green of second morning to do it. ;d
--------------------------------
--------------------------------
Iteration: 1
Original:  Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Adversary:  awww, that's a bummer.  you shoulda gotten david green of second morning to doing it. ;d
--------------------------------
--------------------------------
Iteration: 2
Original:  is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
Adversary:  is worried that he can't access his facebook by texting it... and might cry as a result  school today also. blah!
--------------------------------
--------------------------------
Iteration: 3
Original:  is upset that he can't update his Facebook by texting it... and might cry as a result

In [ ]:
perturbed_data['original_attribution'] = original_attributions
perturbed_data['perturbed_attribution'] = perturbed_attributions
perturbed_data['L1'] = L1s
perturbed_data['L2'] = L2s
perturbed_data['Linf'] = Linfs
perturbed_data['cosine_similarity'] = cos_sims

In [ ]:
labels = [0,0,0,0,1,1,0,0,1,1,1,1,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,1,1,1,1,0,0,1,1,0,0]

In [ ]:
perturbed_data = perturbed_data[:100]
perturbed_data['robust'] = labels

In [ ]:
frame = pd.read_csv('enriched_data.csv')

In [ ]:
def evaluate_metrics(data, metric, threshold):
    # Count for confusion matrix
    if metric == 'cosine_similarity':
        # Need special case for cosine similarity since smaller value implies greater difference
        TP = len(data.query(f"{metric} < {threshold} & robust == 1"))
        FN = len(data.query(f"{metric} >= {threshold} & robust == 1"))
        FP = len(data.query(f"{metric} < {threshold} & robust == 0"))
        TN = len(data.query(f"{metric} >= {threshold} & robust == 0"))
    else:
        TP = len(data.query(f"{metric} > {threshold} & robust == 1"))
        FN = len(data.query(f"{metric} <= {threshold} & robust == 1"))
        FP = len(data.query(f"{metric} > {threshold} & robust == 0"))
        TN = len(data.query(f"{metric} <= {threshold} & robust == 0"))
    # Ensure correct counts
    assert TP + TN + FP + FN == len(data)
    # Collect statistics
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    accuracy = (TP + TN) / len(data)
    return round(precision, 4), round(recall, 4), round(F1, 4), round(accuracy, 4)

In [ ]:
def metric_and_threshold(data, similarity_metrics):
    best_prec = [-1]
    best_rec = [-1]
    best_f1 = [-1]
    best_acc = [-1]
    for metric in similarity_metrics:
        # Generate range for linear search using median of class
        low = np.median(data[data['robust']==0][metric])
        high = np.median(data[data['robust']==1][metric])
        # Linear search with num steps
        for threshold in np.linspace(low, high, num=50):
            precision, recall, F1, accuracy = evaluate_metrics(data, metric, threshold)
            # Update best performances with score, metric, and threshold
            threshold = round(threshold, 4)
            if precision >= best_prec[0]:
                best_prec = (precision, metric, threshold)
            if recall >= best_rec[0]:
                best_rec = (recall, metric, threshold)
            if F1 >= best_f1[0]:
                best_f1 = (F1, metric, threshold)
            if accuracy >= best_acc[0]:
                best_acc = (accuracy, metric, threshold)
    return best_prec, best_rec, best_f1, best_acc

In [ ]:
metric = "Linf"
low = np.median(frame[frame['robust']==0][metric])
high = np.median(frame[frame['robust']==1][metric])
print(low,high)

0.253 0.669


In [ ]:
similarity_metrics = ['L1', 'L2', 'Linf', 'cosine_similarity']
for metric in similarity_metrics:
    precision, recall, f1, accuracy = metric_and_threshold(frame, [metric])
    print(metric)
    print(f"Best precision {precision[0]} at threshold {precision[2]}")
    print(f"Best recall {recall[0]} at threshold {recall[2]}")
    print(f"Best F1 {f1[0]} at threshold {f1[2]}")
    print(f"Best accuracy {accuracy[0]} at threshold {accuracy[2]}")
    print("-------------------------------")

L1
Best precision 0.7097 at threshold 4.3849
Best recall 1.0 at threshold 2.1441
Best F1 0.7358 at threshold 2.1441
Best accuracy 0.76 at threshold 2.9844
-------------------------------
L2
Best precision 0.6897 at threshold 1.6005
Best recall 0.9487 at threshold 0.7235
Best F1 0.7447 at threshold 0.7819
Best accuracy 0.76 at threshold 0.7819
-------------------------------
Linf
Best precision 0.7632 at threshold 0.5417
Best recall 0.8718 at threshold 0.3634
Best F1 0.7532 at threshold 0.5417
Best accuracy 0.81 at threshold 0.5417
-------------------------------
cosine_similarity
Best precision 0.725 at threshold 0.8153
Best recall 0.9231 at threshold 0.8687
Best F1 0.7865 at threshold 0.8628
Best accuracy 0.81 at threshold 0.8509
-------------------------------


In [ ]:
precision, recall, f1, accuracy = metric_and_threshold(frame, similarity_metrics)
print("Overall")
print(f"Best precision {precision[0]} with metric {precision[1]} at threshold {precision[2]}")
print(f"Best recall {recall[0]} with metric {recall[1]} at threshold {recall[2]}")
print(f"Best F1 {f1[0]} with metric {f1[1]} at threshold {f1[2]}")
print(f"Best accuracy {accuracy[0]} with metric {accuracy[1]} at threshold {accuracy[2]}")
print("-------------------------------")

Overall
Best precision 0.7632 with metric Linf at threshold 0.5417
Best recall 1.0 with metric L1 at threshold 2.1441
Best F1 0.7865 with metric cosine_similarity at threshold 0.8628
Best accuracy 0.81 with metric cosine_similarity at threshold 0.8509
-------------------------------


In [ ]:
print(evaluate_metrics(frame, 'cosine_similarity', 0.8628))

(0.7, 0.8974, 0.7865, 0.81)
